In [8]:
# Import libraries
using Turing
using LinearAlgebra
using Distributions
using MultivariateStats
import MultivariateStats: reconstruct
using GaussianProcesses
using StatsBase
using Statistics
using Suppressor
using JLD2
using CSV
using DataFrames, DataFramesMeta
using SplitApplyCombine
using KernelFunctions
using MCMCChains
using PyCall
using PyPlot
using Printf
import PyCall.pyfunction
using Optim

include("Utils/scale_utils.jl")
using .ScaleUtils

include("Utils/gp_utils.jl")
using .GPUtils

# Access the matplotlib module
matplotlib = pyimport("matplotlib")
seaborn = pyimport("seaborn")
os = pyimport("os")
pyimport("scienceplots")
np = pyimport("numpy")
scipy = pyimport("scipy")
skl_model_selection = pyimport("sklearn.model_selection")
plt.style.use(["default","science","no-latex"])
using StatsPlots

seaborn.color_palette("colorblind")

PyCall.pygui(:tk)

# Set a seed for reproducibility
using Random
Random.seed!(11);

SIZE=21
FONTSIZE=21
DOTCOLOR="green"
EDGECOLOR="gray"
plt.style.use(["default","science","no-latex"])
const mpl = PyPlot.matplotlib

PyObject <module 'matplotlib' from '/Users/zachzerbe/anaconda3/envs/brookhaven_julia_env/lib/python3.9/site-packages/matplotlib/__init__.py'>

In [9]:
x = [-1,0,1]
std(x)

1.0

In [ ]:
#For diagnostics

In [7]:
# Assumptions you already have:
# - θ_mat :: Matrix{Float64}  (n_realizations × 6)   # parameters per realization
# - widths :: Vector{Float64}  (n_realizations)      # 90% CI width at chosen_year
# - FONTSIZE, SIZE, EDGECOLOR defined
# - title_dict, tick_dict like in your snippet (LaTeXStrings are fine)
FONTSIZE=27
chosen_year = 2300
cmap = mpl.cm.get_cmap("viridis")
 
# Use explicit endpoints and clip to saturate colors at min/max
vmin, vmax = 9,22.25 # 8.51,16.5 # 1.6,6.3 # 9,22.25
norm_ = mpl.colors.Normalize(vmin=vmin, vmax=vmax, clip="True")
 
output_file_name   = "Plots/Sensitivity_Plots/SLR_sensitivity_wrt_pairwise_parameters_$(chosen_year)_testing.pdf"
 
# Choose a sequential cmap and robust color limits for widths (since widths ≥ 0)
 
# vmin, vmax = quantile(widths, [0.05, 0.95])   # robust limits; change to [minimum(widths), maximum(widths)] if you prefer
# norm_ = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
 
# Build all 15 unique pairs (1..6 choose 2), row-major ordering
pairs = [(a,b) for a in 1:6 for b in a+1:6]  # length = 15
 
# Figure: 5 rows × 3 cols
fig, ax = PyPlot.subplots(nrows=3, ncols=5, figsize=(32, 18), 
    # subplot_kw = Dict("box_aspect" => 1),
    gridspec_kw = Dict(
        "width_ratios"  => [1, 1, 1, 1, 1],
        "height_ratios" => [1, 1, 1],
        # "wspace" => 0.3,
        # "hspace" => 0.35,
    ))
# fig.suptitle("Sea-level Contributions 90% Credible Interval Widths at $(chosen_year)"; fontsize=30, fontweight="bold", y=1.008)
 
# If ax is a Matrix{PyObject}, indexing is ax[row, col]
for (idx, (xk, yk)) in enumerate(pairs)
    rI = 1 + (idx-1) ÷ 5
    cI = 1 + (idx-1) % 5
    axi = ax[rI, cI]
 
    # Scatter: x = param xk, y = param yk, color = widths
    x_var = (xk == 5) ? log.(θ_mat[:, xk]) : θ_mat[:, xk]
    y_var = (yk == 5) ? log.(θ_mat[:, yk]) : θ_mat[:, yk]
 
    axi.set_rasterization_zorder(0)
    sc = axi.scatter(x_var, y_var;
                     s=SIZE*5, c=widths, cmap=cmap, norm=norm_,
                     edgecolor=EDGECOLOR, linewidth=0.5,
                     rasterized=true)
 
    # Labels/ticks/limits
    axi.set_xlabel(string(title_dict[xk]), fontsize=FONTSIZE, labelpad=10)
    axi.set_ylabel(string(title_dict[yk]), fontsize=FONTSIZE, labelpad=10)
    axi.ticklabel_format(style="sci", scilimits=(-3,3), useMathText=true)
    axi.tick_params(axis="both", which="major", labelsize=FONTSIZE*0.8)
    axi.xaxis.offsetText.set_fontsize(FONTSIZE*0.8)
    axi.yaxis.offsetText.set_fontsize(FONTSIZE*0.8)
    axi.locator_params(tight=true, nbins=6)
    axi.grid(true, alpha=0.5, zorder=1)
    axi.set_xticks(tick_dict[xk])
    axi.set_yticks(tick_dict[yk])
    axi.set_xlim(xlim_dict[xk])
    axi.set_ylim(xlim_dict[yk])
end
 
# # Single shared colorbar on the right
# plt.tight_layout(rect=[0.0, 0.0, 0.95, 1.0])  # leave room for colorbar
# cax = fig.add_axes([0.98, 0.25, 0.02, 0.5])  # [left, bottom, width, height]
# sm = mpl.cm.ScalarMappable(norm=norm_, cmap=cmap); sm.set_array([])
# cbar = fig.colorbar(sm, cax=cax)
# cbar.ax.set_title("90% \ncredible \ninterval \nwidth", fontsize=FONTSIZE, pad=30)
# cbar.ax.tick_params(axis="both", which="major", labelsize=FONTSIZE*0.8)
# cbar.mappable.set_clim(8.51,22.5) # 8.51,20.5 # 1.6,6.3
 
# leave a bit of room at the bottom for the horizontal colorbar
plt.tight_layout(rect=[0.0, 0.08, 1.0, 1.0])
 
# horizontal colorbar at the bottom (left, bottom, width, height in figure coords)
cax = fig.add_axes([0.10, 0.04, 0.80, 0.03])
sm = mpl.cm.ScalarMappable(norm=norm_, cmap=cmap)
sm.set_array([])
 
cbar = fig.colorbar(sm, cax=cax, orientation="horizontal")
cbar.set_label("90% credible interval width", fontsize=FONTSIZE*1.2, labelpad=10)
cbar.ax.tick_params(axis="x", which="major", labelsize=FONTSIZE*0.8)
 
mkpath(dirname(output_file_name))
fig.savefig(output_file_name; bbox_inches="tight", dpi=300)

LoadError: UndefVarError: `θ_mat` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [3]:
fig, ax = subplots(figsize=(8, 5.5))
 
selections_2 = 1:1:119 #[1,5,6,10,11,22,25,110,111,112,118,119]
for i in selections_2
    ax.plot(2016:2300, y_grd_vol_raw_mat[i,:], color="salmon", 
                    linewidth=0.8, alpha=0.8, label=i == 1 ? "actual" : "")
end
 
selections_1 = 1:1:100 #[2,13,19,27,40,45,58,70,78,85,94,99,100]
# Plot the time series
for i in selections_1
    ax.plot(2030:15:2300, Y_grd_vol_loaded[i,:], marker = "o", markersize= 6.6, color="black", ls="dashed",
        markerfacecolor="dodgerblue")
#     ax.plot(2030:15:2300, Y_grd_vol_loaded[i,:], color="dodgerblue", 
#                     linewidth=0.8, alpha=0.8, label=i == 1 ? "synthetic" : "")
end
 
 
# Set x-axis and y-axis labels
ax.set_xlabel("Analysis year", fontsize=FONTSIZE, labelpad=10)
ax.set_ylabel("Cumulative grounded mass change",fontsize=FONTSIZE)
 
# Set custom x-axis ticks and labels
ax.ticklabel_format(style="sci", scilimits=(-2,4), useMathText=true)
ax.tick_params(axis="both", which="major", labelsize=FONTSIZE*0.8)
ax.xaxis.offsetText.set_fontsize(FONTSIZE*0.8)
ax.yaxis.offsetText.set_fontsize(FONTSIZE*0.8)
ax.set_xticks([2050, 2100, 2150, 2200, 2250, 2300])
# ax.set_yticks([0, 50, 100, 150, 200])
ax.set_xlim(2030, 2300)
# ax.set_ylim(-25, 235)
ax.grid(true, alpha=0.5, zorder=1)
 
PyPlot.savefig("Synthetic_future_trajectories_all.pdf",bbox_inches="tight",dpi=300)

LoadError: UndefVarError: `subplots` not defined in `Main`
Suggestion: check for spelling errors or missing imports.